In [1]:
%load_ext cython

In [ ]:
%%cython -a
import random    
import math                     
            
def miller_rabin(num,rnds):
    if num == 2 or num == 3 or num == 5 or num == 7 or num == 11:
        return True     
    elif num == 4 or num == 6 or num == 8 or num == 9 or num == 10:
        return False      
    elif num > 2 and rnds > 0:      
        if num % 2 == 0:
            return False            
        else:
            s = 0
            t = num - 1           
            while t % 2 == 0:
                t //= 2
                s +=1           
            for i in range(rnds):
                a = random.randint(2,num-2)
                x = pow(a,t,num)
                if x == 1 or x == num - 1:
                    continue               
                for j in range(s-1):
                    x = pow(x,2,num)
                    if x == num - 1:
                        break
                else:
                    return False
            return True            
    else:
        print('Неверные данные\nПервый аргумент >= 2\nВторой > 0')

In [ ]:
ex1 = 233644041304680072699949795023810733936348917337177642573003025647464227586924845446450732322802680267208401293264931001193901
%%timeit
miller_rabin(ex1,100)

In [ ]:
%%cython -a
import random    
import math                     
            
def miller_rabin(int num, int rnds):
    cdef int t,s,i,a,x,j
    if num == 2 or num == 3 or num == 5 or num == 7 or num == 11:
        return True     
    elif num == 4 or num == 6 or num == 8 or num == 9 or num == 10:
        return False      
    elif num > 2 and rnds > 0:      
        if num % 2 == 0:
            return False            
        else:
            s = 0
            t = num - 1           
            while t % 2 == 0:
                t //= 2
                s +=1           
            for i in range(rnds):
                a = random.randint(2,num-2)
                x = pow(a,t,num)
                if x == 1 or x == num - 1:
                    continue               
                for j in range(s-1):
                    x = pow(x,2,num)
                    if x == num - 1:
                        break
                else:
                    return False
            return True            
    else:
        print('Неверные данные\nПервый аргумент >= 2\nВторой > 0')

In [ ]:
%%timeit
miller_rabin(ex1,100)